In [1]:
# --- Environment and Version Check ---
import sys
import torch
import importlib
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
try:
    import pcdet
    print(f"OpenPCDet version: {getattr(pcdet, '__version__', 'unknown')}")
    print(f"OpenPCDet path: {pcdet.__file__}")
except ImportError:
    print("OpenPCDet not installed!")
try:
    yolov5_spec = importlib.util.find_spec('yolov5')#need to know more on this
    if yolov5_spec is not None:
        import yolov5
        print(f"YOLOv5 path: {yolov5.__file__}")
    else:
        print("YOLOv5 not installed!")
except Exception as e:
    print(f"YOLOv5 import error: {e}")

Python version: 3.9.23 (main, Jun  5 2025, 13:40:20) 
[GCC 11.2.0]
PyTorch version: 1.11.0+cu115
OpenPCDet version: 0.6.0+8caccce+py4c5215f
OpenPCDet path: /home/manas/OpenPCDet/pcdet/__init__.py
YOLOv5 path: /home/manas/miniconda3/envs/sensor_fusion/lib/python3.9/site-packages/yolov5/__init__.py


In [2]:
! pip install yolov5 --quiet

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import Image
import glob
import open3d as o3d
import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
import glob
import os

# Set working directory dynamically
work_dir = os.getcwd()

# Collect image, point cloud, and calibration file paths
image_files = sorted(glob.glob(os.path.join(work_dir, "data/img/*.png")))
point_files = sorted(glob.glob(os.path.join(work_dir, "data/velodyne/*.bin")))
calib_files = sorted(glob.glob(os.path.join(work_dir, "data/calib/*.txt")))

# (Optional) Skip label_files if not available
#label_files = []  # We'll skip using this for now

# Print summary
print("Number of images:", len(image_files))
print("Number of point clouds:", len(point_files))
print("Number of calibration files:", len(calib_files))

Number of images: 320
Number of point clouds: 320
Number of calibration files: 320


In [5]:
import os
import sys
import glob
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
from pathlib import Path
import pickle
from collections import defaultdict
import json

# OpenPCDet imports
try:
    from pcdet.config import cfg, cfg_from_yaml_file
    from pcdet.datasets import DatasetTemplate
    from pcdet.models import build_network, load_data_to_gpu
    from pcdet.utils import common_utils
    print("OpenPCDet imported successfully")
except ImportError as e:
    print(f"OpenPCDet import error: {e}")
    print("Please ensure OpenPCDet is properly installed")

# YOLOv5 imports
try:
    import yolov5
    print("YOLOv5 imported successfully")
except ImportError as e:
    print(f"YOLOv5 import error: {e}")
    print("Please install YOLOv5: pip install yolov5")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# =============================================================================
# CELL 2: Setup Paths and Validate Data
# =============================================================================

# Set working directory
work_dir = os.getcwd()
print(f"Working directory: {work_dir}")

# Data paths
data_dir = os.path.join(work_dir, "data")
image_dir = os.path.join(data_dir, "img")
velodyne_dir = os.path.join(data_dir, "velodyne")
calib_dir = os.path.join(data_dir, "calib")

# Collect file paths
image_files = sorted(glob.glob(os.path.join(image_dir, "*.png")))
point_files = sorted(glob.glob(os.path.join(velodyne_dir, "*.bin")))
calib_files = sorted(glob.glob(os.path.join(calib_dir, "*.txt")))

# Validate data consistency
print("=== Data Validation ===")
print(f"Number of images: {len(image_files)}")
print(f"Number of point clouds: {len(point_files)}")
print(f"Number of calibration files: {len(calib_files)}")

# Check if all counts match
if len(image_files) == len(point_files) == len(calib_files):
    print("✓ All data counts match")
    num_samples = len(image_files)
else:
    print("✗ Data count mismatch!")
    num_samples = min(len(image_files), len(point_files), len(calib_files))
    print(f"Using minimum count: {num_samples}")

OpenPCDet imported successfully
YOLOv5 imported successfully
PyTorch version: 1.11.0+cu115
CUDA available: True
Working directory: /home/manas/Sensor_fusion/Sensor-Fusion
=== Data Validation ===
Number of images: 320
Number of point clouds: 320
Number of calibration files: 320
✓ All data counts match


In [7]:
print("=== Setting up YOLOv5 Camera Detection ===")

# Load YOLOv5 model (using pretrained weights)
try:
    camera_model = yolov5.load('yolov5s.pt')  # You can use yolov5m.pt, yolov5l.pt, yolov5x.pt for better accuracy
    camera_model.eval()
    print("✓ YOLOv5 model loaded successfully")
    
    # Set confidence threshold
    camera_model.conf = 0.4  # confidence threshold
    camera_model.iou = 0.45  # IoU threshold for NMS
    
    # Check device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    camera_model.to(device)
    print(f"✓ YOLOv5 model moved to device: {device}")
    
except Exception as e:
    print(f"✗ Error loading YOLOv5: {e}")
    camera_model = None

# Define COCO class names (YOLOv5 is trained on COCO dataset)
COCO_CLASSES = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck'
]  # need to look more on this


=== Setting up YOLOv5 Camera Detection ===
✓ YOLOv5 model loaded successfully
✓ YOLOv5 model moved to device: cuda


In [8]:
print("=== Setting up PointPillars Model ===")

# Manually specify your config and checkpoint paths
config_path = "/home/manas/OpenPCDet/tools/cfgs/kitti_models/pointpillar.yaml"  # Update if needed
ckpt_path = "/home/manas/Downloads/pointpillar_7728.pth"  # Update if needed

try:
    # Load config
    from pcdet.config import cfg, cfg_from_yaml_file
    cfg_from_yaml_file(config_path, cfg)
    
    # Build model
    from pcdet.models import build_network
    from pcdet.utils import common_utils
    from pcdet.datasets import DatasetTemplate
    dataset = DatasetTemplate(dataset_cfg=cfg.DATA_CONFIG, class_names=cfg.CLASS_NAMES, training=False, root_path=None, logger=common_utils.create_logger())
    lidar_model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=dataset)
    
    # Load pretrained weights
    lidar_model.load_params_from_file(ckpt_path, logger=common_utils.create_logger())
    if torch.cuda.is_available():
        lidar_model.cuda()
    else:
        lidar_model.cpu()
    lidar_model.eval()
    print("✓ PointPillars model initialized with your config and weights")
except Exception as e:
    print(f"✗ Error initializing PointPillars: {e}")
    lidar_model = None

2025-07-29 15:46:43,958   INFO  ==> Loading parameters from checkpoint /home/manas/Downloads/pointpillar_7728.pth to GPU
2025-07-29 15:46:43,958   INFO  ==> Loading parameters from checkpoint /home/manas/Downloads/pointpillar_7728.pth to GPU
2025-07-29 15:46:44,020   INFO  ==> Done (loaded 127/127)
2025-07-29 15:46:44,020   INFO  ==> Done (loaded 127/127)


=== Setting up PointPillars Model ===
✓ PointPillars model initialized with your config and weights


In [9]:
class KITTICalibration:
    """KITTI calibration file parser"""
    
    def __init__(self, calib_file):
        """Load calibration data from file"""
        calibs = self.read_calib_file(calib_file)
        
        # Camera intrinsics and extrinsics
        self.P2 = calibs['P2'].reshape(3, 4)  # Camera 2 projection matrix
        self.R0_rect = calibs['R0_rect'].reshape(3, 3)  # Rectification matrix
        self.Tr_velo_to_cam = calibs['Tr_velo_to_cam'].reshape(3, 4)  # Velodyne to camera
        
        # Create full transformation matrices
        self.R0_rect_4x4 = np.eye(4)
        self.R0_rect_4x4[:3, :3] = self.R0_rect
        
        self.Tr_velo_to_cam_4x4 = np.eye(4)
        self.Tr_velo_to_cam_4x4[:3, :] = self.Tr_velo_to_cam
        
    def read_calib_file(self, filepath):
        """Read KITTI calibration file"""
        data = {}
        with open(filepath, 'r') as f:
            for line in f.readlines():
                line = line.rstrip()
                if len(line) == 0: continue
                key, value = line.split(':', 1)
                try:
                    data[key] = np.array([float(x) for x in value.split()])
                except ValueError:
                    pass
        return data
    
    def project_velo_to_image(self, pts_3d_velo):
        """Project velodyne points to camera image"""
        # Convert to homogeneous coordinates
        pts_3d_velo_homo = np.hstack([pts_3d_velo, np.ones((pts_3d_velo.shape[0], 1))])
        
        # Transform: Velodyne -> Camera -> Rectified Camera -> Image
        pts_3d_cam = pts_3d_velo_homo @ self.Tr_velo_to_cam_4x4.T
        pts_3d_rect = pts_3d_cam @ self.R0_rect_4x4.T
        pts_2d_image = pts_3d_rect @ self.P2.T
        
        # Normalize homogeneous coordinates
        pts_2d_image[:, 0] /= pts_2d_image[:, 2]
        pts_2d_image[:, 1] /= pts_2d_image[:, 2]
        
        return pts_2d_image[:, :2], pts_3d_rect[:, 2]  # Return 2D points and depths

def load_kitti_pointcloud(bin_file):
    """Load KITTI point cloud from binary file"""
    points = np.fromfile(bin_file, dtype=np.float32).reshape(-1, 4)
    return points[:, :3]  # Return only XYZ coordinates

def process_camera_detections(image_path, model):
    """Process camera image and return detections"""
    if model is None:
        return [], None
    
    try:
        # Load and process image
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Run inference
        results = model(image_rgb)
        
        # Extract detections
        detections = []
        if len(results.xyxy[0]) > 0:
            for detection in results.xyxy[0].cpu().numpy():
                x1, y1, x2, y2, conf, cls = detection
                class_name = COCO_CLASSES[int(cls)]
                
                detections.append({
                    'bbox': [x1, y1, x2, y2],
                    'confidence': conf,
                    'class': class_name,
                    'class_id': int(cls)
                })
        
        return detections, image_rgb
    
    except Exception as e:
        print(f"Error in camera detection: {e}")
        return [], None

def prepare_lidar_input(points, dataset):
    """
    Prepare a single point cloud for PointPillars inference using OpenPCDet's dataset utilities.
    """
    # Create input dict as expected by OpenPCDet
    input_dict = {
        'points': points,
        'frame_id': 0,
    }
    # Use the dataset's prepare_data method to get the correct batch dict
    data_dict = dataset.prepare_data(input_dict)
    # Collate into batch (OpenPCDet expects batched input)
    batch_dict = dataset.collate_batch([data_dict])
    return batch_dict

# Update process_lidar_detections to use this utility
def process_lidar_detections(points, model, dataset, device='cuda'):
    """Process LiDAR point cloud and return detections using OpenPCDet pipeline."""
    if model is None or dataset is None:
        return []
    try:
        batch_dict = prepare_lidar_input(points, dataset)
        # Move all tensors to device
        for k in batch_dict:
            if isinstance(batch_dict[k], torch.Tensor):
                batch_dict[k] = batch_dict[k].to(device)
        with torch.no_grad():
            pred_dicts, _ = model(batch_dict)
        detections = []
        if len(pred_dicts) > 0:
            pred_dict = pred_dicts[0]
            if 'pred_boxes' in pred_dict:
                boxes = pred_dict['pred_boxes'].cpu().numpy()
                scores = pred_dict['pred_scores'].cpu().numpy()
                labels = pred_dict['pred_labels'].cpu().numpy()
                for i, (box, score, label) in enumerate(zip(boxes, scores, labels)):
                    if score > 0.3:
                        detections.append({
                            'bbox_3d': box,
                            'confidence': score,
                            'class': cfg.CLASS_NAMES[label-1],
                            'class_id': label-1
                        })
        return detections
    except Exception as e:
        print(f"Error in LiDAR detection: {e}")
        import traceback
        traceback.print_exc()
        return []

In [10]:
class LateFusionProcessor:
    """Late fusion processor for camera and LiDAR detections"""
    
    def __init__(self, camera_model, lidar_model):
        self.camera_model = camera_model
        self.lidar_model = lidar_model
        
        # Fusion parameters
        self.iou_threshold = 0.1
        self.confidence_weights = {'camera': 0.6, 'lidar': 0.6}
        
    def compute_2d_iou(self, box1, box2):
        """Compute IoU between two 2D bounding boxes"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        if x2 <= x1 or y2 <= y1:
            return 0.0
        
        intersection = (x2 - x1) * (y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0.0
    
    def project_3d_to_2d_bbox(self, bbox_3d, calib):
        """Project 3D bounding box to 2D image coordinates"""
        # Extract 3D box parameters (x, y, z, w, l, h, rotation)
        x, y, z, w, l, h, rot = bbox_3d[:7]
        
        # Create 3D box corners
        corners_3d = self.get_3d_box_corners(x, y, z, w, l, h, rot)
        
        # Project to image
        corners_2d, depths = calib.project_velo_to_image(corners_3d)
        
        # Get 2D bounding box
        if len(corners_2d) > 0:
            x_min, y_min = np.min(corners_2d, axis=0)
            x_max, y_max = np.max(corners_2d, axis=0)
            return [x_min, y_min, x_max, y_max]
        
        return None
    
    def get_3d_box_corners(self, x, y, z, w, l, h, rot):
        """Get 3D box corners from center point and dimensions"""
        # Create box corners in object coordinate system
        corners = np.array([
            [-l/2, -w/2, -h/2], [l/2, -w/2, -h/2],
            [l/2, w/2, -h/2], [-l/2, w/2, -h/2],
            [-l/2, -w/2, h/2], [l/2, -w/2, h/2],
            [l/2, w/2, h/2], [-l/2, w/2, h/2]
        ])
        
        # Rotation matrix around Z-axis
        cos_rot = np.cos(rot)
        sin_rot = np.sin(rot)
        rot_matrix = np.array([
            [cos_rot, -sin_rot, 0],
            [sin_rot, cos_rot, 0],
            [0, 0, 1]
        ])
        
        # Rotate and translate
        corners = corners @ rot_matrix.T
        corners[:, 0] += x
        corners[:, 1] += y
        corners[:, 2] += z
        
        return corners
    
    def fuse_detections(self, camera_detections, lidar_detections, calib):
        """Perform late fusion of camera and LiDAR detections"""
        fused_detections = []
        used_lidar = set()
        
        # Process camera detections
        for cam_det in camera_detections:
            best_match = None
            best_iou = 0
            best_lidar_idx = -1
            
            # Find matching LiDAR detection
            for i, lidar_det in enumerate(lidar_detections):
                if i in used_lidar:
                    continue
                
                # Project 3D box to 2D
                bbox_2d = self.project_3d_to_2d_bbox(lidar_det['bbox_3d'], calib)
                
                if bbox_2d is not None:
                    iou = self.compute_2d_iou(cam_det['bbox'], bbox_2d)
                    
                    if iou > best_iou and iou > self.iou_threshold:
                        best_iou = iou
                        best_match = lidar_det
                        best_lidar_idx = i
            
            # Create fused detection
            if best_match is not None:
                # Fuse confidences
                fused_confidence = (
                    self.confidence_weights['camera'] * cam_det['confidence'] +
                    self.confidence_weights['lidar'] * best_match['confidence']
                )
                
                fused_detection = {
                    'bbox_2d': cam_det['bbox'],
                    'bbox_3d': best_match['bbox_3d'],
                    'confidence': fused_confidence,
                    'camera_confidence': cam_det['confidence'],
                    'lidar_confidence': best_match['confidence'],
                    'class': cam_det['class'],
                    'fusion_type': 'camera_lidar',
                    'iou': best_iou
                }
                
                fused_detections.append(fused_detection)
                used_lidar.add(best_lidar_idx)
            else:
                # Camera-only detection
                fused_detection = {
                    'bbox_2d': cam_det['bbox'],
                    'bbox_3d': None,
                    'confidence': cam_det['confidence'],
                    'camera_confidence': cam_det['confidence'],
                    'lidar_confidence': 0.0,
                    'class': cam_det['class'],
                    'fusion_type': 'camera_only',
                    'iou': 0.0
                }
                fused_detections.append(fused_detection)
        
        # Add unmatched LiDAR detections
        for i, lidar_det in enumerate(lidar_detections):
            if i not in used_lidar:
                fused_detection = {
                    'bbox_2d': None,
                    'bbox_3d': lidar_det['bbox_3d'],
                    'confidence': lidar_det['confidence'],
                    'camera_confidence': 0.0,
                    'lidar_confidence': lidar_det['confidence'],
                    'class': lidar_det['class'],
                    'fusion_type': 'lidar_only',
                    'iou': 0.0
                }
                fused_detections.append(fused_detection)
        
        return fused_detections

# Initialize fusion processor
fusion_processor = LateFusionProcessor(camera_model, lidar_model)
print("✓ Late fusion processor initialized")


✓ Late fusion processor initialized


In [11]:
def visualize_detections(image, camera_dets, lidar_dets, fused_dets, calib, save_path=None):
    """Visualize camera, LiDAR, and fused detections"""
    
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    
    # Original image
    axes[0, 0].imshow(image)
    axes[0, 0].set_title('Original Image')
    axes[0, 0].axis('off')
    
    # Camera detections
    img_cam = image.copy()
    for det in camera_dets:
        bbox = det['bbox']
        cv2.rectangle(img_cam, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)
        cv2.putText(img_cam, f"{det['class']}: {det['confidence']:.2f}", 
                   (int(bbox[0]), int(bbox[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    axes[0, 1].imshow(img_cam)
    axes[0, 1].set_title(f'Camera Detections ({len(camera_dets)})')
    axes[0, 1].axis('off')
    
    # LiDAR detections projected to image
    img_lidar = image.copy()
    for det in lidar_dets:
        bbox_2d = fusion_processor.project_3d_to_2d_bbox(det['bbox_3d'], calib)
        if bbox_2d is not None:
            cv2.rectangle(img_lidar, (int(bbox_2d[0]), int(bbox_2d[1])), 
                         (int(bbox_2d[2]), int(bbox_2d[3])), (0, 255, 0), 2)
            cv2.putText(img_lidar, f"{det['class']}: {det['confidence']:.2f}", 
                       (int(bbox_2d[0]), int(bbox_2d[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    axes[1, 0].imshow(img_lidar)
    axes[1, 0].set_title(f'LiDAR Detections Projected ({len(lidar_dets)})')
    axes[1, 0].axis('off')
    
    # Fused detections
    img_fused = image.copy()
    colors = {'camera_lidar': (255, 0, 255), 'camera_only': (255, 0, 0), 'lidar_only': (0, 255, 0)}
    
    for det in fused_dets:
        color = colors[det['fusion_type']]
        
        if det['bbox_2d'] is not None:
            bbox = det['bbox_2d']
            cv2.rectangle(img_fused, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, 2)
            cv2.putText(img_fused, f"{det['class']}: {det['confidence']:.2f} ({det['fusion_type']})", 
                       (int(bbox[0]), int(bbox[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        elif det['bbox_3d'] is not None:
            # Project LiDAR-only detection
            bbox_2d = fusion_processor.project_3d_to_2d_bbox(det['bbox_3d'], calib)
            if bbox_2d is not None:
                cv2.rectangle(img_fused, (int(bbox_2d[0]), int(bbox_2d[1])), 
                             (int(bbox_2d[2]), int(bbox_2d[3])), color, 2)
                cv2.putText(img_fused, f"{det['class']}: {det['confidence']:.2f} ({det['fusion_type']})", 
                           (int(bbox_2d[0]), int(bbox_2d[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    axes[1, 1].imshow(img_fused)
    axes[1, 1].set_title(f'Fused Detections ({len(fused_dets)})')
    axes[1, 1].axis('off')
    
    # Add legend
    legend_elements = [
        plt.Rectangle((0, 0), 1, 1, facecolor='red', alpha=0.7, label='Camera Only'),
        plt.Rectangle((0, 0), 1, 1, facecolor='green', alpha=0.7, label='LiDAR Only'),
        plt.Rectangle((0, 0), 1, 1, facecolor='magenta', alpha=0.7, label='Camera + LiDAR')
    ]
    axes[1, 1].legend(handles=legend_elements, loc='upper right')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"✓ Visualization saved to: {save_path}")
    
    plt.show()

def plot_detection_statistics(results):
    """Plot statistics of detection results"""
    fusion_types = ['camera_only', 'lidar_only', 'camera_lidar']
    type_counts = {ft: 0 for ft in fusion_types}
    confidences = {ft: [] for ft in fusion_types}
    
    for result in results:
        for det in result['fused_detections']:
            fusion_type = det['fusion_type']
            type_counts[fusion_type] += 1
            confidences[fusion_type].append(det['confidence'])
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Detection type distribution
    axes[0].bar(type_counts.keys(), type_counts.values(), 
                color=['red', 'green', 'magenta'], alpha=0.7)
    axes[0].set_title('Detection Type Distribution')
    axes[0].set_ylabel('Count')
    
    # Confidence distribution
    for ft, confs in confidences.items():
        if confs:
            axes[1].hist(confs, alpha=0.7, label=ft, bins=20)
    
    axes[1].set_title('Confidence Distribution by Fusion Type')
    axes[1].set_xlabel('Confidence')
    axes[1].set_ylabel('Count')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

print("✓ Visualization functions defined")

✓ Visualization functions defined


In [12]:
def prepare_lidar_input(points, dataset):
    """
    Prepare a single point cloud for PointPillars inference using OpenPCDet's dataset utilities.
    """
    try:
        # Ensure points are in the correct format (N, 4) with intensity
        if points.shape[1] == 3:
            # Add dummy intensity column if missing
            intensity = np.ones((points.shape[0], 1), dtype=points.dtype)
            points = np.hstack([points, intensity])
        
        # Create input dict as expected by OpenPCDet
        input_dict = {
            'points': points,
            'frame_id': 0,
        }
        
        # Use the dataset's prepare_data method to get the correct batch dict
        data_dict = dataset.prepare_data(input_dict)
        
        # Collate into batch (OpenPCDet expects batched input)
        batch_dict = dataset.collate_batch([data_dict])
        
        return batch_dict
    except Exception as e:
        print(f"Error in prepare_lidar_input: {e}")
        return None

def process_lidar_detections(points, model, dataset, device='cuda'):
    """Process LiDAR point cloud and return detections using OpenPCDet pipeline."""
    if model is None or dataset is None:
        print("Model or dataset is None")
        return []
    
    try:
        # Prepare input data
        batch_dict = prepare_lidar_input(points, dataset)
        if batch_dict is None:
            print("Failed to prepare LiDAR input")
            return []
        
        # Move all tensors to device
        for k in batch_dict:
            if isinstance(batch_dict[k], torch.Tensor):
                batch_dict[k] = batch_dict[k].to(device)
        
        # Run inference
        with torch.no_grad():
            pred_dicts, _ = model(batch_dict)
        
        detections = []
        if len(pred_dicts) > 0:
            pred_dict = pred_dicts[0]
            
            if 'pred_boxes' in pred_dict:
                boxes = pred_dict['pred_boxes'].cpu().numpy()
                scores = pred_dict['pred_scores'].cpu().numpy()
                labels = pred_dict['pred_labels'].cpu().numpy()
                
                # Filter detections by confidence threshold
                confidence_threshold = 0.3
                
                for i, (box, score, label) in enumerate(zip(boxes, scores, labels)):
                    if score > confidence_threshold:
                        # Ensure label index is valid
                        if label > 0 and label <= len(cfg.CLASS_NAMES):
                            class_name = cfg.CLASS_NAMES[label-1]
                        else:
                            class_name = f"class_{label}"
                        
                        detections.append({
                            'bbox_3d': box,
                            'confidence': float(score),
                            'class': class_name,
                            'class_id': int(label-1)
                        })
        
        print(f"PointPillars detected {len(detections)} objects")
        return detections
        
    except Exception as e:
        print(f"Error in LiDAR detection: {e}")
        import traceback
        traceback.print_exc()
        return []

def process_single_sample_fixed(sample_idx=3):
    """Fixed version of process_single_sample with proper dataset passing"""
    
    if sample_idx >= num_samples:
        print(f"Error: Sample index {sample_idx} out of range (0-{num_samples-1})")
        return None
    
    print(f"=== Processing Sample {sample_idx} ===")
    
    # Load data paths
    image_path = image_files[sample_idx]
    point_path = point_files[sample_idx]
    calib_path = calib_files[sample_idx]
    
    print(f"Image: {os.path.basename(image_path)}")
    print(f"Point cloud: {os.path.basename(point_path)}")
    print(f"Calibration: {os.path.basename(calib_path)}")
    
    try:
        # Load calibration
        calib = KITTICalibration(calib_path)
        print("✓ Calibration loaded")
        
        # Load point cloud
        points = load_kitti_pointcloud(point_path)
        print(f"✓ Point cloud loaded: {points.shape[0]} points")
        
        # Process camera detections
        print("Processing camera detections...")
        camera_detections, image = process_camera_detections(image_path, camera_model)
        print(f"✓ Camera detections: {len(camera_detections)}")
        
        # Process LiDAR detections (fixed with dataset parameter)
        print("Processing LiDAR detections...")
        lidar_detections = process_lidar_detections(points, lidar_model, dataset)  # Added dataset parameter
        print(f"✓ LiDAR detections: {len(lidar_detections)}")
        
        # Perform fusion
        print("Performing late fusion...")
        fused_detections = fusion_processor.fuse_detections(
            camera_detections, lidar_detections, calib
        )
        print(f"✓ Fused detections: {len(fused_detections)}")
        
        # Print detection details
        print("\n=== Detection Details ===")
        print("Camera detections:")
        for i, det in enumerate(camera_detections):
            print(f"  {i+1}. {det['class']} (conf: {det['confidence']:.3f})")
        
        print("LiDAR detections:")
        for i, det in enumerate(lidar_detections):
            print(f"  {i+1}. {det['class']} (conf: {det['confidence']:.3f})")
            
        print("Fused detections:")
        for i, det in enumerate(fused_detections):
            print(f"  {i+1}. {det['class']} (conf: {det['confidence']:.3f}, type: {det['fusion_type']})")
        
        # Visualize results
        if image is not None:
            visualize_detections(
                image, camera_detections, lidar_detections, fused_detections, calib,
                save_path=f"fusion_result_sample_{sample_idx}.png"
            )
        
        return {
            'sample_idx': sample_idx,
            'camera_detections': camera_detections,
            'lidar_detections': lidar_detections,
            'fused_detections': fused_detections,
            'image': image,
            'points': points,
            'calibration': calib
        }
        
    except Exception as e:
        print(f"✗ Error processing sample {sample_idx}: {e}")
        import traceback
        traceback.print_exc()
        return None

# Additional debugging function to check model and dataset
def debug_pointpillars_setup():
    """Debug function to check PointPillars model and dataset setup"""
    print("=== PointPillars Debug Info ===")
    
    # Check model
    if lidar_model is None:
        print("✗ LiDAR model is None")
        return False
    else:
        print("✓ LiDAR model loaded")
        print(f"  Model type: {type(lidar_model)}")
        print(f"  Model device: {next(lidar_model.parameters()).device}")
    
    # Check dataset
    if 'dataset' not in globals():
        print("✗ Dataset not found in globals")
        return False
    else:
        print("✓ Dataset available")
        print(f"  Dataset type: {type(dataset)}")
        print(f"  Class names: {cfg.CLASS_NAMES}")
    
    # Check config
    print(f"✓ Config loaded")
    print(f"  Number of classes: {len(cfg.CLASS_NAMES)}")
    print(f"  Classes: {cfg.CLASS_NAMES}")
    
    return True

# Test the debug function
print("Running PointPillars debug check...")
debug_result = debug_pointpillars_setup()

if debug_result:
    print("\n✓ PointPillars setup looks good. You can now run:")
    print("test_result = process_single_sample_fixed(3)")
else:
    print("\n✗ Issues found with PointPillars setup. Please check the errors above.")

Running PointPillars debug check...
=== PointPillars Debug Info ===
✓ LiDAR model loaded
  Model type: <class 'pcdet.models.detectors.pointpillar.PointPillar'>
  Model device: cuda:0
✓ Dataset available
  Dataset type: <class 'pcdet.datasets.dataset.DatasetTemplate'>
  Class names: ['Car', 'Pedestrian', 'Cyclist']
✓ Config loaded
  Number of classes: 3
  Classes: ['Car', 'Pedestrian', 'Cyclist']

✓ PointPillars setup looks good. You can now run:
test_result = process_single_sample_fixed(3)


In [13]:
import torch
import numpy as np

def prepare_lidar_input_fixed(points, dataset):
    """
    Fixed version: Prepare a single point cloud for PointPillars inference with proper tensor handling.
    """
    try:
        # Ensure points are in the correct format (N, 4) with intensity
        if points.shape[1] == 3:
            # Add dummy intensity column if missing
            intensity = np.ones((points.shape[0], 1), dtype=points.dtype)
            points = np.hstack([points, intensity])
        
        # Ensure points are float32
        points = points.astype(np.float32)
        
        # Create input dict as expected by OpenPCDet
        input_dict = {
            'points': points,
            'frame_id': 0,
        }
        
        # Use the dataset's prepare_data method to get the correct batch dict
        data_dict = dataset.prepare_data(input_dict)
        
        # Convert numpy arrays to tensors before collating
        for key, value in data_dict.items():
            if isinstance(value, np.ndarray):
                data_dict[key] = torch.from_numpy(value)
        
        # Collate into batch (OpenPCDet expects batched input)
        batch_dict = dataset.collate_batch([data_dict])
        
        return batch_dict
        
    except Exception as e:
        print(f"Error in prepare_lidar_input_fixed: {e}")
        import traceback
        traceback.print_exc()
        return None

def process_lidar_detections_fixed(points, model, dataset, device='cuda'):
    """Fixed version: Process LiDAR point cloud with proper tensor handling."""
    if model is None or dataset is None:
        print("Model or dataset is None")
        return []
    
    try:
        # Prepare input data
        batch_dict = prepare_lidar_input_fixed(points, dataset)
        if batch_dict is None:
            print("Failed to prepare LiDAR input")
            return []
        
        print(f"Batch dict keys: {list(batch_dict.keys())}")
        
        # Move all tensors to device and ensure they are tensors
        device_obj = torch.device(device)
        for k, v in batch_dict.items():
            if isinstance(v, torch.Tensor):
                batch_dict[k] = v.to(device_obj)
                print(f"Moved {k} to {device_obj}, shape: {v.shape}, dtype: {v.dtype}")
            elif isinstance(v, np.ndarray):
                # Convert remaining numpy arrays to tensors
                batch_dict[k] = torch.from_numpy(v).to(device_obj)
                print(f"Converted and moved {k} to {device_obj}, shape: {v.shape}")
            elif isinstance(v, list):
                # Handle lists (e.g., batch_size info)
                print(f"List field {k}: {v}")
        
        # Run inference
        model.eval()  # Ensure model is in eval mode
        with torch.no_grad():
            pred_dicts, _ = model(batch_dict)
        
        detections = []
        if len(pred_dicts) > 0:
            pred_dict = pred_dicts[0]
            
            if 'pred_boxes' in pred_dict and len(pred_dict['pred_boxes']) > 0:
                boxes = pred_dict['pred_boxes'].cpu().numpy()
                scores = pred_dict['pred_scores'].cpu().numpy()
                labels = pred_dict['pred_labels'].cpu().numpy()
                
                print(f"Raw predictions: {len(boxes)} boxes, score range: [{scores.min():.3f}, {scores.max():.3f}]")
                
                # Filter detections by confidence threshold
                confidence_threshold = 0.3
                
                for i, (box, score, label) in enumerate(zip(boxes, scores, labels)):
                    if score > confidence_threshold:
                        # Ensure label index is valid
                        if label > 0 and label <= len(cfg.CLASS_NAMES):
                            class_name = cfg.CLASS_NAMES[label-1]
                        else:
                            class_name = f"class_{label}"
                        
                        detections.append({
                            'bbox_3d': box,
                            'confidence': float(score),
                            'class': class_name,
                            'class_id': int(label-1)
                        })
            else:
                print("No 'pred_boxes' in prediction or empty predictions")
        
        print(f"PointPillars detected {len(detections)} objects (after filtering)")
        return detections
        
    except Exception as e:
        print(f"Error in LiDAR detection: {e}")
        import traceback
        traceback.print_exc()
        return []

def process_single_sample_v2(sample_idx=3):
    """Version 2 of process_single_sample with fixed LiDAR processing"""
    
    if sample_idx >= num_samples:
        print(f"Error: Sample index {sample_idx} out of range (0-{num_samples-1})")
        return None
    
    print(f"=== Processing Sample {sample_idx} (v2) ===")
    
    # Load data paths
    image_path = image_files[sample_idx]
    point_path = point_files[sample_idx]
    calib_path = calib_files[sample_idx]
    
    print(f"Image: {os.path.basename(image_path)}")
    print(f"Point cloud: {os.path.basename(point_path)}")
    print(f"Calibration: {os.path.basename(calib_path)}")
    
    try:
        # Load calibration
        calib = KITTICalibration(calib_path)
        print("✓ Calibration loaded")
        
        # Load point cloud
        points = load_kitti_pointcloud(point_path)
        print(f"✓ Point cloud loaded: {points.shape[0]} points")
        
        # Process camera detections
        print("Processing camera detections...")
        camera_detections, image = process_camera_detections(image_path, camera_model)
        print(f"✓ Camera detections: {len(camera_detections)}")
        
        # Process LiDAR detections with fixed function
        print("Processing LiDAR detections (v2)...")
        lidar_detections = process_lidar_detections_fixed(points, lidar_model, dataset)
        print(f"✓ LiDAR detections: {len(lidar_detections)}")
        
        # Perform fusion
        print("Performing late fusion...")
        fused_detections = fusion_processor.fuse_detections(
            camera_detections, lidar_detections, calib
        )
        print(f"✓ Fused detections: {len(fused_detections)}")
        
        # Print detection details
        print("\n=== Detection Details ===")
        print("Camera detections:")
        for i, det in enumerate(camera_detections):
            print(f"  {i+1}. {det['class']} (conf: {det['confidence']:.3f})")
        
        print("LiDAR detections:")
        for i, det in enumerate(lidar_detections):
            print(f"  {i+1}. {det['class']} (conf: {det['confidence']:.3f})")
            
        print("Fused detections:")
        for i, det in enumerate(fused_detections):
            print(f"  {i+1}. {det['class']} (conf: {det['confidence']:.3f}, type: {det['fusion_type']})")
        
        # Visualize results
        if image is not None:
            visualize_detections(
                image, camera_detections, lidar_detections, fused_detections, calib,
                save_path=f"fusion_result_sample_{sample_idx}_v2.png"
            )
        
        return {
            'sample_idx': sample_idx,
            'camera_detections': camera_detections,
            'lidar_detections': lidar_detections,
            'fused_detections': fused_detections,
            'image': image,
            'points': points,
            'calibration': calib
        }
        
    except Exception as e:
        print(f"✗ Error processing sample {sample_idx}: {e}")
        import traceback
        traceback.print_exc()
        return None

# Test with the fixed version
print("=== Testing Fixed LiDAR Processing ===")
test_result_v2 = process_single_sample_v2(3)

=== Testing Fixed LiDAR Processing ===
=== Processing Sample 3 (v2) ===
Image: 000003.png
Point cloud: 000003.bin
Calibration: 000003.txt
✓ Calibration loaded
✓ Point cloud loaded: 121795 points
Processing camera detections...
✓ Camera detections: 4
Processing LiDAR detections (v2)...
Batch dict keys: ['points', 'frame_id', 'lidar_aug_matrix', 'use_lead_xyz', 'voxels', 'voxel_coords', 'voxel_num_points', 'batch_size']
Converted and moved points to cuda, shape: (63289, 5)
Converted and moved frame_id to cuda, shape: (1,)
Converted and moved lidar_aug_matrix to cuda, shape: (1, 4, 4)
Converted and moved use_lead_xyz to cuda, shape: (1,)
Converted and moved voxels to cuda, shape: (8494, 32, 4)
Converted and moved voxel_coords to cuda, shape: (8494, 4)
Converted and moved voxel_num_points to cuda, shape: (8494,)
voxel_features.shape: torch.Size([8494, 32, 4])
voxel_num_points.shape: torch.Size([8494])
Raw predictions: 20 boxes, score range: [0.104, 0.394]
PointPillars detected 1 objects (a

In [ ]:
#!/usr/bin/env python3
"""
Complete Late Fusion System for Camera and LiDAR Object Detection
Combines YOLOv5 (camera) and PointPillars (LiDAR) detections using late fusion approach
"""

import os
import sys
import glob
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
from pathlib import Path
import pickle
from collections import defaultdict
import json
import warnings
warnings.filterwarnings('ignore')

# OpenPCDet imports
try:
    from pcdet.config import cfg, cfg_from_yaml_file
    from pcdet.datasets import DatasetTemplate
    from pcdet.models import build_network, load_data_to_gpu
    from pcdet.utils import common_utils
    print("✓ OpenPCDet imported successfully")
except ImportError as e:
    print(f"✗ OpenPCDet import error: {e}")
    print("Please ensure OpenPCDet is properly installed")

# YOLOv5 imports
try:
    import yolov5
    print("✓ YOLOv5 imported successfully")
except ImportError as e:
    print(f"✗ YOLOv5 import error: {e}")
    print("Please install YOLOv5: pip install yolov5")

class KITTICalibration:
    """KITTI calibration file parser for coordinate transformations"""
    
    def __init__(self, calib_file):
        """Load calibration data from KITTI calibration file"""
        calibs = self.read_calib_file(calib_file)
        
        # Camera intrinsics and extrinsics
        self.P2 = calibs['P2'].reshape(3, 4)  # Camera 2 projection matrix
        self.R0_rect = calibs['R0_rect'].reshape(3, 3)  # Rectification matrix
        self.Tr_velo_to_cam = calibs['Tr_velo_to_cam'].reshape(3, 4)  # Velodyne to camera
        
        # Create full 4x4 transformation matrices
        self.R0_rect_4x4 = np.eye(4)
        self.R0_rect_4x4[:3, :3] = self.R0_rect
        
        self.Tr_velo_to_cam_4x4 = np.eye(4)
        self.Tr_velo_to_cam_4x4[:3, :] = self.Tr_velo_to_cam
        
    def read_calib_file(self, filepath):
        """Read KITTI calibration file and parse parameters"""
        data = {}
        with open(filepath, 'r') as f:
            for line in f.readlines():
                line = line.rstrip()
                if len(line) == 0: 
                    continue
                key, value = line.split(':', 1)
                try:
                    data[key] = np.array([float(x) for x in value.split()])
                except ValueError:
                    pass
        return data
    
    def project_velo_to_image(self, pts_3d_velo):
        """Project Velodyne 3D points to camera image coordinates"""
        # Convert to homogeneous coordinates
        pts_3d_velo_homo = np.hstack([pts_3d_velo, np.ones((pts_3d_velo.shape[0], 1))])
        
        # Transform: Velodyne -> Camera -> Rectified Camera -> Image
        pts_3d_cam = pts_3d_velo_homo @ self.Tr_velo_to_cam_4x4.T
        pts_3d_rect = pts_3d_cam @ self.R0_rect_4x4.T
        pts_2d_image = pts_3d_rect @ self.P2.T
        
        # Normalize homogeneous coordinates
        valid_mask = pts_2d_image[:, 2] > 0
        pts_2d_image[valid_mask, 0] /= pts_2d_image[valid_mask, 2]
        pts_2d_image[valid_mask, 1] /= pts_2d_image[valid_mask, 2]
        
        return pts_2d_image[:, :2], pts_3d_rect[:, 2]  # Return 2D points and depths

class CameraDetector:
    """YOLOv5-based camera object detector"""
    
    def __init__(self, model_name='yolov5s', confidence_threshold=0.4, iou_threshold=0.45):
        """Initialize YOLOv5 model"""
        self.model_name = model_name
        self.confidence_threshold = confidence_threshold
        self.iou_threshold = iou_threshold
        
        # Load YOLOv5 model
        try:
            self.model = yolov5.load(f'{model_name}.pt')
            self.model.eval()
            self.model.conf = confidence_threshold
            self.model.iou = iou_threshold
            
            # Move to GPU if available
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            self.model.to(self.device)
            print(f"✓ YOLOv5 model loaded and moved to {self.device}")
            
        except Exception as e:
            print(f"✗ Error loading YOLOv5: {e}")
            self.model = None
    
    def detect(self, image_path):
        """Detect objects in camera image"""
        if self.model is None:
            return [], None
        
        try:
            # Load and process image
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Could not load image {image_path}")
                return [], None
                
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Run inference
            results = self.model(image_rgb)
            
            # Extract detections
            detections = []
            if len(results.xyxy[0]) > 0:
                for detection in results.xyxy[0].cpu().numpy():
                    x1, y1, x2, y2, conf, cls = detection
                    class_name = results.names[int(cls)]
                    
                    detections.append({
                        'bbox': [float(x1), float(y1), float(x2), float(y2)],
                        'confidence': float(conf),
                        'class': class_name,
                        'class_id': int(cls)
                    })
            
            return detections, image_rgb
        
        except Exception as e:
            print(f"Error in camera detection: {e}")
            return [], None

class LiDARDetector:
    """PointPillars-based LiDAR object detector"""
    
    def __init__(self, config_path, checkpoint_path, confidence_threshold=0.3):
        """Initialize PointPillars model"""
        self.config_path = config_path
        self.checkpoint_path = checkpoint_path
        self.confidence_threshold = confidence_threshold
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        try:
            # Load config
            cfg_from_yaml_file(config_path, cfg)
            
            # Build model
            self.dataset = DatasetTemplate(
                dataset_cfg=cfg.DATA_CONFIG, 
                class_names=cfg.CLASS_NAMES, 
                training=False, 
                root_path=None, 
                logger=common_utils.create_logger()
            )
            
            self.model = build_network(
                model_cfg=cfg.MODEL, 
                num_class=len(cfg.CLASS_NAMES), 
                dataset=self.dataset
            )
            
            # Load pretrained weights
            self.model.load_params_from_file(checkpoint_path, logger=common_utils.create_logger())
            self.model.to(self.device)
            self.model.eval()
            
            print(f"✓ PointPillars model loaded on {self.device}")
            
        except Exception as e:
            print(f"✗ Error initializing PointPillars: {e}")
            self.model = None
            self.dataset = None
    
    def load_pointcloud(self, bin_file):
        """Load KITTI point cloud from binary file"""
        points = np.fromfile(bin_file, dtype=np.float32).reshape(-1, 4)
        return points
    
    def prepare_input(self, points):
        """Prepare point cloud for PointPillars inference"""
        try:
            # Ensure points are float32 and have intensity channel
            points = points.astype(np.float32)
            if points.shape[1] == 3:
                # Add dummy intensity if missing
                intensity = np.ones((points.shape[0], 1), dtype=np.float32)
                points = np.hstack([points, intensity])
            
            # Create input dict
            input_dict = {
                'points': points,
                'frame_id': 0,
            }
            
            # Prepare data using dataset
            data_dict = self.dataset.prepare_data(input_dict)
            
            # Convert numpy arrays to tensors
            for key, value in data_dict.items():
                if isinstance(value, np.ndarray):
                    data_dict[key] = torch.from_numpy(value)
            
            # Collate into batch
            batch_dict = self.dataset.collate_batch([data_dict])
            
            # Move to device
            for k, v in batch_dict.items():
                if isinstance(v, torch.Tensor):
                    batch_dict[k] = v.to(self.device)
                elif isinstance(v, np.ndarray):
                    batch_dict[k] = torch.from_numpy(v).to(self.device)
            
            return batch_dict
            
        except Exception as e:
            print(f"Error preparing LiDAR input: {e}")
            return None
    
    def detect(self, point_file):
        """Detect objects in LiDAR point cloud"""
        if self.model is None or self.dataset is None:
            return []
        
        try:
            # Load point cloud
            points = self.load_pointcloud(point_file)
            
            # Prepare input
            batch_dict = self.prepare_input(points)
            if batch_dict is None:
                return []
            
            # Run inference
            with torch.no_grad():
                pred_dicts, _ = self.model(batch_dict)
            
            # Extract detections
            detections = []
            if len(pred_dicts) > 0:
                pred_dict = pred_dicts[0]
                
                if 'pred_boxes' in pred_dict and len(pred_dict['pred_boxes']) > 0:
                    boxes = pred_dict['pred_boxes'].cpu().numpy()
                    scores = pred_dict['pred_scores'].cpu().numpy()
                    labels = pred_dict['pred_labels'].cpu().numpy()
                    
                    # Filter by confidence threshold
                    for box, score, label in zip(boxes, scores, labels):
                        if score > self.confidence_threshold:
                            # Ensure valid label
                            if 0 < label <= len(cfg.CLASS_NAMES):
                                class_name = cfg.CLASS_NAMES[label-1]
                            else:
                                class_name = f"class_{label}"
                            
                            detections.append({
                                'bbox_3d': box,
                                'confidence': float(score),
                                'class': class_name,
                                'class_id': int(label-1)
                            })
            
            return detections
            
        except Exception as e:
            print(f"Error in LiDAR detection: {e}")
            return []

class LateFusionProcessor:
    """Late fusion processor for camera and LiDAR detections"""
    
    def __init__(self, iou_threshold=0.1, camera_weight=0.6, lidar_weight=0.6):
        """Initialize fusion processor with parameters"""
        self.iou_threshold = iou_threshold
        self.camera_weight = camera_weight
        self.lidar_weight = lidar_weight
        
    def compute_2d_iou(self, box1, box2):
        """Compute IoU between two 2D bounding boxes"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        if x2 <= x1 or y2 <= y1:
            return 0.0
        
        intersection = (x2 - x1) * (y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0.0
    
    def get_3d_box_corners(self, x, y, z, w, l, h, rot):
        """Get 3D bounding box corners from center and dimensions"""
        # Create box corners in object coordinate system
        corners = np.array([
            [-l/2, -w/2, -h/2], [l/2, -w/2, -h/2],
            [l/2, w/2, -h/2], [-l/2, w/2, -h/2],
            [-l/2, -w/2, h/2], [l/2, -w/2, h/2],
            [l/2, w/2, h/2], [-l/2, w/2, h/2]
        ])
        
        # Rotation matrix around Z-axis
        cos_rot = np.cos(rot)
        sin_rot = np.sin(rot)
        rot_matrix = np.array([
            [cos_rot, -sin_rot, 0],
            [sin_rot, cos_rot, 0],
            [0, 0, 1]
        ])
        
        # Rotate and translate
        corners = corners @ rot_matrix.T
        corners[:, 0] += x
        corners[:, 1] += y
        corners[:, 2] += z
        
        return corners
    
    def project_3d_to_2d_bbox(self, bbox_3d, calib):
        """Project 3D bounding box to 2D image coordinates"""
        try:
            # Extract 3D box parameters
            x, y, z, w, l, h, rot = bbox_3d[:7]
            
            # Get 3D box corners
            corners_3d = self.get_3d_box_corners(x, y, z, w, l, h, rot)
            
            # Project to image
            corners_2d, depths = calib.project_velo_to_image(corners_3d)
            
            # Filter points behind camera
            valid_mask = depths > 0
            if not np.any(valid_mask):
                return None
            
            valid_corners = corners_2d[valid_mask]
            
            # Get 2D bounding box
            if len(valid_corners) > 0:
                x_min, y_min = np.min(valid_corners, axis=0)
                x_max, y_max = np.max(valid_corners, axis=0)
                return [x_min, y_min, x_max, y_max]
            
        except Exception as e:
            print(f"Error projecting 3D box: {e}")
        
        return None
    
    def fuse_detections(self, camera_detections, lidar_detections, calibration):
        """Perform late fusion of camera and LiDAR detections"""
        fused_detections = []
        used_lidar = set()
        
        # Process camera detections and find matching LiDAR detections
        for cam_det in camera_detections:
            best_match = None
            best_iou = 0
            best_lidar_idx = -1
            
            # Find best matching LiDAR detection
            for i, lidar_det in enumerate(lidar_detections):
                if i in used_lidar:
                    continue
                
                # Project 3D box to 2D
                bbox_2d = self.project_3d_to_2d_bbox(lidar_det['bbox_3d'], calibration)
                
                if bbox_2d is not None:
                    iou = self.compute_2d_iou(cam_det['bbox'], bbox_2d)
                    
                    if iou > best_iou and iou > self.iou_threshold:
                        best_iou = iou
                        best_match = lidar_det
                        best_lidar_idx = i
            
            # Create fused detection
            if best_match is not None:
                # Fuse confidences
                fused_confidence = (
                    self.camera_weight * cam_det['confidence'] +
                    self.lidar_weight * best_match['confidence']
                )
                
                fused_detection = {
                    'bbox_2d': cam_det['bbox'],
                    'bbox_3d': best_match['bbox_3d'],
                    'confidence': fused_confidence,
                    'camera_confidence': cam_det['confidence'],
                    'lidar_confidence': best_match['confidence'],
                    'class': cam_det['class'],
                    'fusion_type': 'camera_lidar',
                    'iou': best_iou
                }
                
                fused_detections.append(fused_detection)
                used_lidar.add(best_lidar_idx)
            else:
                # Camera-only detection
                fused_detection = {
                    'bbox_2d': cam_det['bbox'],
                    'bbox_3d': None,
                    'confidence': cam_det['confidence'],
                    'camera_confidence': cam_det['confidence'],
                    'lidar_confidence': 0.0,
                    'class': cam_det['class'],
                    'fusion_type': 'camera_only',
                    'iou': 0.0
                }
                fused_detections.append(fused_detection)
        
        # Add unmatched LiDAR detections
        for i, lidar_det in enumerate(lidar_detections):
            if i not in used_lidar:
                fused_detection = {
                    'bbox_2d': None,
                    'bbox_3d': lidar_det['bbox_3d'],
                    'confidence': lidar_det['confidence'],
                    'camera_confidence': 0.0,
                    'lidar_confidence': lidar_det['confidence'],
                    'class': lidar_det['class'],
                    'fusion_type': 'lidar_only',
                    'iou': 0.0
                }
                fused_detections.append(fused_detection)
        
        return fused_detections

class VisualizationManager:
    """Handle visualization of detection results"""
    
    def __init__(self):
        self.colors = {
            'camera_lidar': (255, 0, 255),  # Magenta
            'camera_only': (255, 0, 0),     # Red
            'lidar_only': (0, 255, 0)       # Green
        }
    
    def draw_2d_bbox(self, image, bbox, color, label, thickness=2):
        """Draw 2D bounding box on image"""
        x1, y1, x2, y2 = [int(coord) for coord in bbox]
        cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
        
        # Add label
        label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
        cv2.rectangle(image, (x1, y1 - label_size[1] - 10), 
                     (x1 + label_size[0], y1), color, -1)
        cv2.putText(image, label, (x1, y1 - 5), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    def visualize_detections(self, image, camera_dets, lidar_dets, fused_dets, 
                           calibration, fusion_processor, save_path=None):
        """Create comprehensive visualization of all detections"""
        
        fig, axes = plt.subplots(2, 2, figsize=(20, 16))
        
        # Original image
        axes[0, 0].imshow(image)
        axes[0, 0].set_title('Original Image', fontsize=14)
        axes[0, 0].axis('off')
        
        # Camera detections
        img_cam = image.copy()
        for det in camera_dets:
            label = f"{det['class']}: {det['confidence']:.2f}"
            self.draw_2d_bbox(img_cam, det['bbox'], self.colors['camera_only'], label)
        
        axes[0, 1].imshow(img_cam)
        axes[0, 1].set_title(f'Camera Detections ({len(camera_dets)})', fontsize=14)
        axes[0, 1].axis('off')
        
        # LiDAR detections projected to image
        img_lidar = image.copy()
        for det in lidar_dets:
            bbox_2d = fusion_processor.project_3d_to_2d_bbox(det['bbox_3d'], calibration)
            if bbox_2d is not None:
                label = f"{det['class']}: {det['confidence']:.2f}"
                self.draw_2d_bbox(img_lidar, bbox_2d, self.colors['lidar_only'], label)
        
        axes[1, 0].imshow(img_lidar)
        axes[1, 0].set_title(f'LiDAR Detections Projected ({len(lidar_dets)})', fontsize=14)
        axes[1, 0].axis('off')
        
        # Fused detections
        img_fused = image.copy()
        for det in fused_dets:
            color = self.colors[det['fusion_type']]
            
            if det['bbox_2d'] is not None:
                label = f"{det['class']}: {det['confidence']:.2f} ({det['fusion_type']})"
                self.draw_2d_bbox(img_fused, det['bbox_2d'], color, label)
            elif det['bbox_3d'] is not None:
                # Project LiDAR-only detection
                bbox_2d = fusion_processor.project_3d_to_2d_bbox(det['bbox_3d'], calibration)
                if bbox_2d is not None:
                    label = f"{det['class']}: {det['confidence']:.2f} ({det['fusion_type']})"
                    self.draw_2d_bbox(img_fused, bbox_2d, color, label)
        
        axes[1, 1].imshow(img_fused)
        axes[1, 1].set_title(f'Fused Detections ({len(fused_dets)})', fontsize=14)
        axes[1, 1].axis('off')
        
        # Add legend
        from matplotlib.patches import Patch
        legend_elements = [
            Patch(facecolor='red', alpha=0.7, label='Camera Only'),
            Patch(facecolor='green', alpha=0.7, label='LiDAR Only'),
            Patch(facecolor='magenta', alpha=0.7, label='Camera + LiDAR')
        ]
        axes[1, 1].legend(handles=legend_elements, loc='upper right')
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            print(f"✓ Visualization saved to: {save_path}")
        
        plt.show()

class LateFusionSystem:
    """Main system class that orchestrates the complete late fusion pipeline"""
    
    def __init__(self, config):
        """Initialize the complete fusion system"""
        self.config = config
        
        # Initialize components
        self.camera_detector = CameraDetector(
            model_name=config.get('camera_model', 'yolov5s'),
            confidence_threshold=config.get('camera_confidence', 0.4)
        )
        
        self.lidar_detector = LiDARDetector(
            config_path=config['lidar_config_path'],
            checkpoint_path=config['lidar_checkpoint_path'],
            confidence_threshold=config.get('lidar_confidence', 0.3)
        )
        
        self.fusion_processor = LateFusionProcessor(
            iou_threshold=config.get('iou_threshold', 0.1),
            camera_weight=config.get('camera_weight', 0.6),
            lidar_weight=config.get('lidar_weight', 0.6)
        )
        
        self.visualizer = VisualizationManager()
        
        print("✓ Late Fusion System initialized")
    
    def process_sample(self, image_path, pointcloud_path, calibration_path, 
                      visualize=True, save_path=None):
        """Process a single sample with camera and LiDAR data"""
        
        try:
            print(f"=== Processing Sample ===")
            print(f"Image: {os.path.basename(image_path)}")
            print(f"Point cloud: {os.path.basename(pointcloud_path)}")
            print(f"Calibration: {os.path.basename(calibration_path)}")
            
            # Load calibration
            calibration = KITTICalibration(calibration_path)
            print("✓ Calibration loaded")
            
            # Camera detection
            print("Processing camera detections...")
            camera_detections, image = self.camera_detector.detect(image_path)
            print(f"✓ Camera detections: {len(camera_detections)}")
            
            # LiDAR detection
            print("Processing LiDAR detections...")
            lidar_detections = self.lidar_detector.detect(pointcloud_path)
            print(f"✓ LiDAR detections: {len(lidar_detections)}")
            
            # Fusion
            print("Performing late fusion...")
            fused_detections = self.fusion_processor.fuse_detections(
                camera_detections, lidar_detections, calibration
            )
            print(f"✓ Fused detections: {len(fused_detections)}")
            
            # Print detailed results
            self.print_detection_summary(camera_detections, lidar_detections, fused_detections)
            
            # Visualization
            if visualize and image is not None:
                self.visualizer.visualize_detections(
                    image, camera_detections, lidar_detections, fused_detections,
                    calibration, self.fusion_processor, save_path
                )
            
            return {
                'camera_detections': camera_detections,
                'lidar_detections': lidar_detections,
                'fused_detections': fused_detections,
                'image': image,
                'calibration': calibration
            }
            
        except Exception as e:
            print(f"✗ Error processing sample: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    def print_detection_summary(self, camera_dets, lidar_dets, fused_dets):
        """Print detailed summary of detections"""
        print("\n=== Detection Summary ===")
        
        print("Camera detections:")
        for i, det in enumerate(camera_dets):
            print(f"  {i+1}. {det['class']} (confidence: {det['confidence']:.3f})")
        
        print("LiDAR detections:")
        for i, det in enumerate(lidar_dets):
            print(f"  {i+1}. {det['class']} (confidence: {det['confidence']:.3f})")
        
        print("Fused detections:")
        for i, det in enumerate(fused_dets):
            print(f"  {i+1}. {det['class']} (confidence: {det['confidence']:.3f}, "
                  f"type: {det['fusion_type']}, IoU: {det['iou']:.3f})")
    
    def process_dataset(self, data_directory, max_samples=None, save_results=True):
        """Process multiple samples from a dataset directory"""
        
        # Find data files
        image_files = sorted(glob.glob(os.path.join(data_directory, "img", "*.png")))
        point_files = sorted(glob.glob(os.path.join(data_directory, "velodyne", "*.bin")))
        calib_files = sorted(glob.glob(os.path.join(data_directory, "calib", "*.txt")))
        
        # Validate data consistency
        num_samples = min(len(image_files), len(point_files), len(calib_files))
        if max_samples:
            num_samples = min(num_samples, max_samples)
        
        print(f"\n=== Processing Dataset ===")
        print(f"Found {len(image_files)} images, {len(point_files)} point clouds, "
              f"{len(calib_files)} calibration files")
        print(f"Processing {num_samples} samples...")
        
        results = []
        for i in range(num_samples):
            print(f"\n--- Sample {i+1}/{num_samples} ---")
            
            result = self.process_sample(
                image_files[i], point_files[i], calib_files[i],
                visualize=False,  # Don't visualize during batch processing
                save_path=f"fusion_result_{i:06d}.png" if save_results else None
            )
            
            if result:
                results.append(result)
        
        print(f"\n✓ Processed {len(results)} samples successfully")
        return results

# Example usage and configuration
def main():
    """Main function demonstrating system usage"""
    
    # Configuration
    config = {
        'camera_model': 'yolov5s',
        'camera_confidence': 0.4,
        'lidar_config_path': '/path/to/pointpillar.yaml',  # Update this path
        'lidar_checkpoint_path': '/path/to/pointpillar.pth',  # Update this path
        'lidar_confidence': 0.3,
        'iou_threshold': 0.1,
        'camera_weight': 0.6,
        'lidar_weight': 0.6
    }
    
    # Initialize system
    fusion_system = LateFusionSystem(config)
    
    # Process single sample
    image_path = "data/img/000000.png"
    pointcloud_path = "data/velodyne/000000.bin"
    calibration_path = "data/calib/000000.txt"
    
    result = fusion_system.process_sample(
        image_path, pointcloud_path, calibration_path,
        visualize=True, save_path="fusion_result.png"
    )
    
    # Process entire dataset (optional)
    # results = fusion_system.process_dataset("data", max_samples=10)

if __name__ == "__main__":
    main()


# Additional utility functions for advanced usage

def create_detection_statistics(results):
    """Analyze detection statistics across multiple samples"""
    
    fusion_types = ['camera_only', 'lidar_only', 'camera_lidar']
    type_counts = {ft: 0 for ft in fusion_types}
    confidences = {ft: [] for ft in fusion_types}
    class_counts = defaultdict(int)
    
    for result in results:
        for det in result['fused_detections']:
            fusion_type = det['fusion_type']
            type_counts[fusion_type] += 1
            confidences[fusion_type].append(det['confidence'])
            class_counts[det['class']] += 1
    
    # Plot statistics
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Detection type distribution
    axes[0, 0].bar(type_counts.keys(), type_counts.values(), 
                   color=['red', 'green', 'magenta'], alpha=0.7)
    axes[0, 0].set_title('Detection Type Distribution')
    axes[0, 0].set_ylabel('Count')
    
    # Confidence distribution
    for ft, confs in confidences.items():
        if confs:
            axes[0, 1].hist(confs, alpha=0.7, label=ft, bins=20)
    axes[0, 1].set_title('Confidence Distribution by Fusion Type')
    axes[0, 1].set_xlabel('Confidence')
    axes[0, 1].set_ylabel('Count')
    axes[0, 1].legend()
    
    # Class distribution
    classes = list(class_counts.keys())
    counts = list(class_counts.values())
    axes[1, 0].bar(classes, counts, alpha=0.7)
    axes[1, 0].set_title('Object Class Distribution')
    axes[1, 0].set_ylabel('Count')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Fusion effectiveness (IoU distribution for fused detections)
    ious = []
    for result in results:
        for det in result['fused_detections']:
            if det['fusion_type'] == 'camera_lidar' and det['iou'] > 0:
                ious.append(det['iou'])
    
    if ious:
        axes[1, 1].hist(ious, bins=20, alpha=0.7, color='purple')
        axes[1, 1].set_title('IoU Distribution for Fused Detections')
        axes[1, 1].set_xlabel('IoU')
        axes[1, 1].set_ylabel('Count')
    else:
        axes[1, 1].text(0.5, 0.5, 'No fused detections', 
                       ha='center', va='center', transform=axes[1, 1].transAxes)
        axes[1, 1].set_title('IoU Distribution for Fused Detections')
    
    plt.tight_layout()
    plt.show()
    
    return {
        'type_counts': type_counts,
        'confidences': confidences,
        'class_counts': dict(class_counts),
        'ious': ious
    }

def save_results_to_file(results, output_path):
    """Save detection results to JSON file"""
    
    # Convert numpy arrays to lists for JSON serialization
    serializable_results = []
    
    for result in results:
        serializable_result = {
            'camera_detections': [],
            'lidar_detections': [],
            'fused_detections': []
        }
        
        # Camera detections
        for det in result['camera_detections']:
            serializable_result['camera_detections'].append({
                'bbox': det['bbox'],
                'confidence': float(det['confidence']),
                'class': det['class'],
                'class_id': int(det['class_id'])
            })
        
        # LiDAR detections
        for det in result['lidar_detections']:
            bbox_3d = det['bbox_3d'].tolist() if isinstance(det['bbox_3d'], np.ndarray) else det['bbox_3d']
            serializable_result['lidar_detections'].append({
                'bbox_3d': bbox_3d,
                'confidence': float(det['confidence']),
                'class': det['class'],
                'class_id': int(det['class_id'])
            })
        
        # Fused detections
        for det in result['fused_detections']:
            fused_det = {
                'bbox_2d': det['bbox_2d'],
                'confidence': float(det['confidence']),
                'camera_confidence': float(det['camera_confidence']),
                'lidar_confidence': float(det['lidar_confidence']),
                'class': det['class'],
                'fusion_type': det['fusion_type'],
                'iou': float(det['iou'])
            }
            
            if det['bbox_3d'] is not None:
                bbox_3d = det['bbox_3d'].tolist() if isinstance(det['bbox_3d'], np.ndarray) else det['bbox_3d']
                fused_det['bbox_3d'] = bbox_3d
            else:
                fused_det['bbox_3d'] = None
                
            serializable_result['fused_detections'].append(fused_det)
        
        serializable_results.append(serializable_result)
    
    # Save to JSON
    with open(output_path, 'w') as f:
        json.dump(serializable_results, f, indent=2)
    
    print(f"✓ Results saved to {output_path}")

def load_results_from_file(input_path):
    """Load detection results from JSON file"""
    
    with open(input_path, 'r') as f:
        data = json.load(f)
    
    print(f"✓ Results loaded from {input_path}")
    return data

class BenchmarkEvaluator:
    """Evaluate fusion system performance against ground truth"""
    
    def __init__(self, iou_threshold=0.5):
        self.iou_threshold = iou_threshold
    
    def compute_metrics(self, predictions, ground_truth):
        """Compute precision, recall, and F1 score"""
        
        tp = 0  # True positives
        fp = 0  # False positives
        fn = 0  # False negatives
        
        # Match predictions with ground truth
        matched_gt = set()
        
        for pred in predictions:
            best_iou = 0
            best_gt_idx = -1
            
            for i, gt in enumerate(ground_truth):
                if i in matched_gt:
                    continue
                
                # Compute IoU (simplified for 2D boxes)
                if pred.get('bbox_2d') and gt.get('bbox'):
                    iou = self.compute_2d_iou(pred['bbox_2d'], gt['bbox'])
                    if iou > best_iou and pred['class'] == gt['class']:
                        best_iou = iou
                        best_gt_idx = i
            
            if best_iou >= self.iou_threshold:
                tp += 1
                matched_gt.add(best_gt_idx)
            else:
                fp += 1
        
        fn = len(ground_truth) - len(matched_gt)
        
        # Calculate metrics
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        return {
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'true_positives': tp,
            'false_positives': fp,
            'false_negatives': fn
        }
    
    def compute_2d_iou(self, box1, box2):
        """Compute IoU between two 2D boxes"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        if x2 <= x1 or y2 <= y1:
            return 0.0
        
        intersection = (x2 - x1) * (y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0.0

# Configuration templates for different scenarios

KITTI_CONFIG = {
    'camera_model': 'yolov5s',
    'camera_confidence': 0.4,
    'lidar_config_path': '/path/to/kitti_pointpillar.yaml',
    'lidar_checkpoint_path': '/path/to/pointpillar_kitti.pth',
    'lidar_confidence': 0.3,
    'iou_threshold': 0.1,
    'camera_weight': 0.6,
    'lidar_weight': 0.6
}

 

def create_config_for_dataset(dataset_name):
    """Create configuration for specific dataset"""
    
    configs = {
        'kitti': KITTI_CONFIG,
        'nuscenes': NUSC_CONFIG
    }
    
    if dataset_name.lower() in configs:
        return configs[dataset_name.lower()].copy()
    else:
        print(f"Warning: Unknown dataset {dataset_name}, using default KITTI config")
        return KITTI_CONFIG.copy()

# Example of advanced usage with custom configuration
def advanced_example():
    """Advanced example showing custom configuration and evaluation"""
    
    # Create custom configuration
    custom_config = {
        'camera_model': 'yolov5m',  # Use medium model for better accuracy
        'camera_confidence': 0.5,
        'lidar_config_path': '/home/manas/OpenPCDet/tools/cfgs/kitti_models/pointpillar.yaml',
        'lidar_checkpoint_path': '/home/manas/Downloads/pointpillar_7728.pth',
        'lidar_confidence': 0.4,
        'iou_threshold': 0.15,  # Higher threshold for stricter matching
        'camera_weight': 0.7,   # Give more weight to camera
        'lidar_weight': 0.5
    }
    
    # Initialize system
    fusion_system = LateFusionSystem(custom_config)
    
    # Process dataset
    data_dir = "data"
    results = fusion_system.process_dataset(data_dir, max_samples=5)
    
    if results:
        # Analyze statistics
        stats = create_detection_statistics(results)
        
        # Save results
        save_results_to_file(results, "fusion_results.json")
        
        # Print summary
        print(f"\n=== Final Summary ===")
        print(f"Total samples processed: {len(results)}")
        print(f"Detection type distribution: {stats['type_counts']}")
        print(f"Average IoU for fused detections: {np.mean(stats['ious']):.3f}" if stats['ious'] else "No fused detections")

# Quick start function for easy setup
def quick_start(data_directory, config_paths):
    """Quick start function for immediate usage"""
    
    # Update paths in config
    config = KITTI_CONFIG.copy()
    config.update(config_paths)
    
    # Initialize and run
    fusion_system = LateFusionSystem(config)
    
    # Find first sample
    image_files = sorted(glob.glob(os.path.join(data_directory, "img", "*.png")))
    point_files = sorted(glob.glob(os.path.join(data_directory, "velodyne", "*.bin")))
    calib_files = sorted(glob.glob(os.path.join(data_directory, "calib", "*.txt")))
    
    if image_files and point_files and calib_files:
        result = fusion_system.process_sample(
            image_files[0], point_files[0], calib_files[0],
            visualize=True, save_path="quick_start_result.png"
        )
        print("✓ Quick start completed successfully!")
        return result
    else:
        print("✗ Could not find data files in the specified directory")
        return None

# Instructions for usage
"""
USAGE INSTRUCTIONS:

1. Basic Usage:
   
   config_paths = {
       'lidar_config_path': '/path/to/your/pointpillar.yaml',
       'lidar_checkpoint_path': '/path/to/your/pointpillar.pth'
   }
   
   result = quick_start("data", config_paths)

2. Advanced Usage:
   
   config = create_config_for_dataset('kitti')
   config['lidar_config_path'] = '/your/path/pointpillar.yaml'
   config['lidar_checkpoint_path'] = '/your/path/pointpillar.pth'
   
   fusion_system = LateFusionSystem(config)
   results = fusion_system.process_dataset("data", max_samples=10)
   
   # Analyze results
   stats = create_detection_statistics(results)
   save_results_to_file(results, "results.json")

3. Required Files:
   - YOLOv5 model (downloaded automatically)
   - PointPillars config file (.yaml)
   - PointPillars checkpoint file (.pth)
   - Data in KITTI format:
     ├── data/
     │   ├── img/           # Camera images (.png)
     │   ├── velodyne/      # LiDAR point clouds (.bin)
     │   └── calib/         # Calibration files (.txt)

4. Dependencies:
   pip install torch torchvision yolov5 opencv-python matplotlib numpy
   
   # OpenPCDet installation (follow official guide)
   git clone https://github.com/open-mmlab/OpenPCDet.git
   cd OpenPCDet
   pip install -v -e .
"""

✓ OpenPCDet imported successfully
✓ YOLOv5 imported successfully
✓ YOLOv5 model loaded and moved to cuda
✗ Error initializing PointPillars: [Errno 2] No such file or directory: '/path/to/pointpillar.yaml'
✓ Late Fusion System initialized
=== Processing Sample ===
Image: 000000.png
Point cloud: 000000.bin
Calibration: 000000.txt
✓ Calibration loaded
Processing camera detections...
✓ Camera detections: 1
Processing LiDAR detections...
✓ LiDAR detections: 0
Performing late fusion...
✓ Fused detections: 1

=== Detection Summary ===
Camera detections:
  1. car (confidence: 0.565)
LiDAR detections:
Fused detections:
  1. car (confidence: 0.565, type: camera_only, IoU: 0.000)
✓ Visualization saved to: fusion_result.png


'\nUSAGE INSTRUCTIONS:\n\n1. Basic Usage:\n   \n   config_paths = {\n       \'lidar_config_path\': \'/path/to/your/pointpillar.yaml\',\n       \'lidar_checkpoint_path\': \'/path/to/your/pointpillar.pth\'\n   }\n   \n   result = quick_start("data", config_paths)\n\n2. Advanced Usage:\n   \n   config = create_config_for_dataset(\'kitti\')\n   config[\'lidar_config_path\'] = \'/your/path/pointpillar.yaml\'\n   config[\'lidar_checkpoint_path\'] = \'/your/path/pointpillar.pth\'\n   \n   fusion_system = LateFusionSystem(config)\n   results = fusion_system.process_dataset("data", max_samples=10)\n   \n   # Analyze results\n   stats = create_detection_statistics(results)\n   save_results_to_file(results, "results.json")\n\n3. Required Files:\n   - YOLOv5 model (downloaded automatically)\n   - PointPillars config file (.yaml)\n   - PointPillars checkpoint file (.pth)\n   - Data in KITTI format:\n     ├── data/\n     │   ├── img/           # Camera images (.png)\n     │   ├── velodyne/      # Li